<a href="https://colab.research.google.com/github/tmiller5/CS3080FinalProjectTrevorMillerBderKhan/blob/main/PokemonSpriteGeneratorGPT2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Special thanks to Mathew Rayfield who created the original model for this project

In [ ]:
!rm -rf /content/checkpoint/

In [ ]:
import sys
import math
from PIL import Image
from os import listdir
import shutil
import os
import time
import re


%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files, drive

gpt2.download_gpt2(model_name="124M")

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



Fetching checkpoint: 1.05Mit [00:00, 269Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 35.3Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 622Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 178Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 655Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 126Mit/s]                                                 
Fetching vocab.bpe: 1.05Mit [00:00, 175Mit/s]                                                       


This is where we mount our google drive for reading and writing to and from VVVV

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


Converting the images for training to a gpt-2 acceptable format

In [ ]:
output = ''
input_folder = 'emerald-pokemon-sprites' #@param {type:"string"}
input_folder_path = '/content/drive/My Drive/%s' % input_folder

for f in listdir(input_folder_path):
    ext = f.split('.')[-1]
    if ext not in ['png', 'gif', 'jpg', 'jpeg']: #Ensures the extension of the file is of a certain type
      continue

    path = '%s/%s' % (input_folder_path, f)
    image = Image.open(path).convert('RGBA')
    pixels = image.load()
    width = image.size[0]
    height = image.size[1]

    lines = []

    for y in range(height):
        padLength = len(str(width))
        padded = str(y).zfill(padLength)
        split = [padded + 'd']

        for x in range(width):
            color = pixels[x, y] 

            s = '~'

            if len(color) < 4 or color[3] > 128:
                r = color[0]
                g = color[1]
                b = color[2]
               
                if r == 85 and g == 85 and b == 85:
                    s = 'a' # use grayscale character 1

                elif r == 170 and g == 170 and b == 170:
                    s = 'b' # use grayscale character 2

                else: # use one of 64 color characters
                    mR = math.floor(r/64)
                    mG = math.floor(g/64)
                    mB = math.floor(b/64)

                    c = 0;

                    c += mR
                    c = c << 2
                    c += mG
                    c = c << 2
                    c += mB

                    s = chr(c+33)

            split.append(s)

        lines.append(' '.join(split))

    reversed = []
    for line in lines:
        reversed.insert(0, (line.replace('d ', 'u ', 1)))

    output += '\n'.join(lines)
    output += '\n'.join(reversed)

print('saving output')
with open('all-images.txt', 'w') as output_file:
    output_file.write(output)
print('done !')

saving output
done !


Training the model:

In [ ]:
checkpoint_folder = 'pokemon-gpt-2-checkpoints'  #@param {type:"string"}
run_name = 'pokemon-gpt-2-run' #@param {type:"string"}
steps = 5000 #@param {type:"integer"}
session = gpt2.start_tf_sess()

session = gpt2.reset_session(session)
gpt2.finetune(session,
    dataset='all-images.txt',
    model_name='124M',
    steps=steps,
    restore_from='latest',
    run_name=run_name,
    print_every=10,
    sample_every=500,
    save_every=500
  )

shutil.copytree('checkpoint/%s' % run_name, '/content/drive/My Drive/%s/%s' % (checkpoint_folder, run_name))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Loading checkpoint models/124M/model.ckpt
INFO:tensorflow:Restoring parameters from models/124M/model.ckpt


  0%|          | 0/1 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 1/1 [00:14<00:00, 14.22s/it]


dataset has 3592506 tokens
Training...
[10 | 28.27] loss=0.59 avg=0.59
[20 | 50.04] loss=0.57 avg=0.58
[30 | 72.13] loss=0.51 avg=0.56
[40 | 94.52] loss=0.55 avg=0.56
[50 | 117.14] loss=0.56 avg=0.56
[60 | 140.07] loss=0.49 avg=0.54
[70 | 163.33] loss=0.40 avg=0.52
[80 | 186.97] loss=0.48 avg=0.52
[90 | 210.69] loss=0.35 avg=0.50
[100 | 234.23] loss=0.45 avg=0.49
[110 | 257.79] loss=0.33 avg=0.48
[120 | 281.51] loss=0.53 avg=0.48
[130 | 305.24] loss=0.66 avg=0.50
[140 | 328.92] loss=0.55 avg=0.50
[150 | 352.59] loss=0.53 avg=0.50
[160 | 376.21] loss=0.38 avg=0.49
[170 | 399.85] loss=0.76 avg=0.51
[180 | 423.52] loss=0.42 avg=0.51
[190 | 447.25] loss=0.14 avg=0.49
[200 | 470.97] loss=0.40 avg=0.48
[210 | 494.67] loss=0.54 avg=0.48
[220 | 518.39] loss=0.39 avg=0.48
[230 | 542.10] loss=0.46 avg=0.48
[240 | 565.80] loss=0.56 avg=0.48
[250 | 589.50] loss=0.33 avg=0.48
[260 | 613.20] loss=0.29 avg=0.47
[270 | 636.93] loss=0.51 avg=0.47
[280 | 660.63] loss=0.40 avg=0.47
[290 | 684.33] loss=0.

FileExistsError: ignored

In [ ]:
checkpoint_folder = 'pokemon-gpt-2-checkpoints'  #@param {type:"string"}
run_name = 'pokemon-gpt-2-run' #@param {type:"string"}

if not os.path.exists('checkpoint/%s' % run_name):
    shutil.copytree('/content/drive/My Drive/%s/%s' % (checkpoint_folder, run_name), 'checkpoint/%s' % run_name)

The code below generates sprites and saves them to a folder in your google drive in the OUTPUT_FOLDER folder. The temperature slider is what determines how crazy the generated sprites are. Somewhere in the middle is usually the best value. 

In [ ]:
output_folder = 'pokemon-gpt-2-output' #@param {type:"string"}
generate_count = 10 #@param {type:"integer"}
temperature = 0.95 #@param {type:"slider", min:0.7, max:1.3, step:0.01}
width = 64 #@param {type:"integer"}
height = 64 #@param {type:"integer"}
save_texts = False #@param {type:"boolean"}

def createImage(text, path):
    lines = text.split('\n')

    imageWidth = 0
    imageHeight = 0

    for line in lines:
        split = line.split(' ')

        marker = split[0]
        match = re.search("^([0-9]+)([du])$", marker)

        if match:
            groups = match.groups()
            index = int(groups[0])

            width = len(split) - 1
            height = index + 1

            if width > imageWidth:
                imageWidth = width;

            if height > imageHeight:
                imageHeight = height

    pixels = []
    for y in range(imageHeight):
        for x in range(imageWidth):
            pixels.append((0, 0, 0, 0))

    for line in lines:
        split = line.split(' ')

        marker = split[0]
        match = re.search("^([0-9]+)([du])$", marker)

        if match:
            groups = match.groups()
            index = int(groups[0])

            for x in range(len(split) - 1):
                s = split[x + 1]

                if s != '~':
                    r = 0
                    g = 0
                    b = 0
                    
                    if s == 'a':
                        r = g = b = 107
                    elif s == 'b':
                        r = g = b = 187
                    else:
                        c = ord(s[0]) - 33

                        b = (c & 3) * 64
                        if b == 192:
                            b += 63

                        c = c >> 2
                        g = (c & 3) * 64
                        if g == 192:
                            g += 63

                        c = c >> 2
                        r = (c & 3) * 64
                        if r == 192:
                            r += 63

                    i = (index * imageWidth) + x

                    pixels[i] = (r, g, b, 255)

    image = Image.new('RGBA', (imageWidth, imageHeight))
    image.putdata(pixels)
    image.save(path)

def blankLines():
    lines = []

    for i in range(0, height):
        lines.append('')

    return lines

sess = None
sess = gpt2.start_tf_sess()
sess = gpt2.reset_session(sess)
for ii in range(0, generate_count):
    print(ii)
    
    if not sess:
        sess = gpt2.start_tf_sess()
    else:
        sess = gpt2.reset_session(sess)
    
    gpt2.load_gpt2(sess, run_name=run_name)

    lines = blankLines()
    prefix = ''
    hasColor = False

    while True:
        text = gpt2.generate(sess, run_name=run_name, prefix=prefix, temperature=temperature, return_as_list=True)[0]

        print('\n\noutput:');
        print(text)

        newLines = text.split('\n')

        direction = None
        lastIndex = None
        for line in newLines:
                split = line.split(' ')[:width + 2]

                if len(split) < 55:
                    break;

                marker = split[0]
                match = re.search("^([0-9]+)([du])$", marker)

                if match:
                    groups = match.groups()

                    try:
                        index = int(groups[0])
                    except:
                        break

                    if direction == None:
                        direction = groups[1]

                    if groups[1] != direction:
                        print('direction changed')
                        break

                    if lastIndex != None:
                        if groups[1] == 'd' and index <= lastIndex:
                            print('bad line order')
                            break
                        elif groups[1] == 'u' and index >= lastIndex:
                            print('bad line order')
                            break
                    lastIndex = index

                    split[0] = marker.replace('u', 'd')

                    if not hasColor:
                        for character in split[2:]:
                            if character != '~' and character != '`':
                                hasColor = True
                                break

                    while len(split) < width:
                        split.append('~')

                    try:
                        lines[index] = ' '.join(split)
                    except IndexError:
                        break

        if not hasColor:
            print('no color')
            lines = blankLines()
            continue

        topIndex = None
        for i in range(0, height):
            if lines[i]:
                topIndex = i
                break

        bottomIndex = None
        for i in range(topIndex, height):
            if lines[i]:
                bottomIndex = i
            else:
                break

        print('\n\ntop %i bottom %i' % (topIndex, bottomIndex))

        sectionSize = 5
        if topIndex > 0:
            section = lines[topIndex:min(topIndex+sectionSize+1, bottomIndex+1)]
            section.reverse()
            for i in range(0, len(section)):
                section[i] = section[i].replace('d', 'u')

        elif bottomIndex < height - 1:
            section = lines[max(bottomIndex-sectionSize, topIndex):bottomIndex+1]

        else:
            print('\n'.join(lines))
            filename = '%i' % int(time.time())
            text = '\n'.join(lines)

            if not os.path.exists('/content/drive/My Drive/%s' % output_folder):
                os.makedirs('/content/drive/My Drive/%s' % output_folder)

            if save_texts:
                text_file = open('/content/drive/My Drive/%s/%s.txt' % (output_folder, filename), 'w')
                text_file.write(text)
                text_file.close()
            
            createImage(text, '/content/drive/My Drive/%s/%s.png' % (output_folder, filename))

            print('saved !')
            break

        prefix = '\n'.join(section)
        print('\n\nprefix:\n%s' % prefix)

0
Loading checkpoint checkpoint/pokemon-gpt-2-run/model-4039
INFO:tensorflow:Restoring parameters from checkpoint/pokemon-gpt-2-run/model-4039


output:
51d ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ! ! Z Z Z E ! F F ! ! ~ A [ [ [ [ A ^ ^ ^ ] ! ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
52d ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ! ! ! ^ ^ _ _ _ _ _ _ _ _ ! ~ ~ A [ [ ! ! ! ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
53d ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ! A Z Z ! ^ ^ _ _ _ _ _ _ _ _ _ ! ~ ~ A [ [ [ [ A ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
54d ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ A A ~ ~ ! A _ Z [ ! [ _ _ _ _ _ _ _ _ _ _ _ ! ~ ~ A [ [ [ [ [ ! ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
55d ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ A A ` [ A ! _ _ 5 [ [ [ _ _ _ _ _ _ _ _ _ _ _ ! ~ ~ ~ A [ [ [ [ [ [ ! ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
56d ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ A A ` ` [ A 5 ` ` [ 5 [ [ [ [ [ [ [ [ [ [ [ [ [ A ~ ~ ~ ~ A A A [ [ [ [ ! ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~
57d ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ ~ A Z ` ` ` ` ` ` 5 [ [